In [1]:
import numpy as np
import time as Time
from numba import jit
import matplotlib.pyplot as plt
import pyvista as pv
import torch

In [14]:
import pstats
from pstats import SortKey
p = pstats.Stats('restats')
p.strip_dirs().sort_stats(-1).print_stats()

Tue Sep 17 11:45:22 2024    restats

         5228532 function calls (4923073 primitive calls) in 1230.466 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      130    0.000    0.000    0.000    0.000 <frozen _collections_abc>:1022(__iter__)
        2    0.000    0.000    0.000    0.000 <frozen _collections_abc>:104(_check_methods)
        1    0.000    0.000    0.000    0.000 <frozen _collections_abc>:288(__subclasshook__)
       50    0.000    0.000    0.000    0.000 <frozen _collections_abc>:341(__subclasshook__)
        6    0.000    0.000    0.000    0.000 <frozen _collections_abc>:435(__subclasshook__)
     2612    0.003    0.000    0.012    0.000 <frozen _collections_abc>:811(__contains__)
       32    0.000    0.000    0.000    0.000 <frozen _collections_abc>:819(keys)
       64    0.000    0.000    0.000    0.000 <frozen _collections_abc>:823(items)
       96    0.000    0.000    0.000    0.000 <frozen _collections

In [18]:
p.strip_dirs().sort_stats(1).print_stats()

Tue Sep 17 11:45:22 2024    restats

         5228532 function calls (4923073 primitive calls) in 1230.466 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      200  358.505    1.793  358.691    1.793 Multi_Species_bosch_etching.py:198(boundary)
      200  288.643    1.443  738.756    3.694 Multi_Species_bosch_etching.py:354(getAcc_depo)
      200  178.555    0.893  310.112    1.551 Multi_Species_bosch_etching.py:253(etching_film)
 7533/511   97.509    0.013  278.540    0.545 {method 'acquire' of '_thread.lock' objects}
      200   76.791    0.384   76.880    0.384 Multi_Species_bosch_etching.py:374(Parcelgen)
     2954   69.130    0.023   69.130    0.023 {method 'astype' of 'numpy.ndarray' objects}
    25288   47.251    0.002   47.251    0.002 {method 'reduce' of 'numpy.ufunc' objects}
      499   42.462    0.085   48.126    0.096 _kdtree.py:353(__init__)
      499   18.504    0.037   20.724    0.042 _kdtree.py:363(query)


In [2]:
%load_ext line_profiler

In [3]:
film = np.zeros((100, 100, 200, 2))

bottom = 100
height = 140

density = 10

sphere = np.ones((100, 100, 200), dtype=bool)

radius = 30

center = 50
for i in range(sphere.shape[0]):
    for j in range(sphere.shape[1]):
        if np.abs(i-center)*np.abs(i-center) + np.abs(j-center)*np.abs(j-center) < radius*radius:
            sphere[i, j, bottom:height] = 0

film[sphere, 1] = density
film[:, :, height:, :] = 0
film[:, :, 0:bottom, 0] = density # bottom
film[:, :, 0:bottom, 1] = 0 # bottom
# film[:, :30, bottom:height, 1] = 10
# film[:, -30:, bottom:height, 1] = 10


In [3]:
sumFilm = np.sum(film, axis=-1)

print(sumFilm.shape)

(100, 100, 200)


In [4]:
depo1 = torch.Tensor(film[:, 50:, :, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(film[:, 50:, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:62984/index.html?ui=P_0x25581e9b9b0_0&reconnect=auto" class="pyvis…

In [8]:
etchfilm = film

In [5]:
etchfilm = np.load('bosch_sf_step10.npy')

In [58]:
indice_c4 = etchfilm[:, :, :, 1] >= 10
etchfilm[indice_c4, 1] = 10

In [6]:
depo1 = torch.Tensor(etchfilm[:50, :, :, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etchfilm[:50, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:62984/index.html?ui=P_0x2558e1a8fe0_1&reconnect=auto" class="pyvis…

In [62]:
etchfilm[50, 55, 45:]

array([[10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [ 0., 10.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,

In [7]:
import Multi_Species_bosch_etching

In [8]:
def max_velocity_u( random1, random2):
    return np.sqrt(-np.log(random1))*(np.cos(2*np.pi*random2))

def max_velocity_w( random1, random2):
    return np.sqrt(-np.log(random1))*(np.sin(2*np.pi*random2))

def max_velocity_v( random3):
    return -np.sqrt(-np.log(random3))

In [9]:
# F+ Ar+ 

N = int(5e7)
velosity_matrix = np.zeros((N, 3))
# tstep=1e-5
# celllength=1e-5
# velosity_matrix[:, 0] = -1 * celllength /tstep
# velosity_matrix[:, 1] = -1 * celllength /tstep
Random1 = np.random.rand(N)
Random2 = np.random.rand(N)
Random3 = np.random.rand(N)
velosity_matrix = np.array([max_velocity_u(Random1, Random2), \
                            max_velocity_w(Random1, Random2), \
                                max_velocity_v(Random3)]).T

energy = np.linalg.norm(velosity_matrix, axis=1)
velosity_matrix[:,0] = np.divide(velosity_matrix[:,0], energy)
velosity_matrix[:,1] = np.divide(velosity_matrix[:,1], energy)
velosity_matrix[:,2] = np.divide(velosity_matrix[:,2], energy)

# velosity_matrix[:, 2] = -1 * celllength /tstep

typeID = np.zeros(N)
# FO_ratio = int(N/4)
# typeID[-FO_ratio:] = 1

# add Ar+
# ion_ration = int(N/4)
# typeID[-ion_ration:] = 2
# velosity_matrix[-ion_ration:, 0] = np.random.rand(ion_ration)*0.01 - 0.005
# velosity_matrix[-ion_ration:, 1] = np.random.rand(ion_ration)*0.01 - 0.005
# velosity_matrix[-ion_ration:, 2] = -1 
# add Ar+


vel_type_shuffle = np.zeros((N, 4))
vel_type_shuffle[:, :3] = velosity_matrix
vel_type_shuffle[:, -1] = typeID

np.random.shuffle(vel_type_shuffle)
# print(typeID[:10])
# print(velosity_matrix[0])
print(vel_type_shuffle[:10])

[[-0.11207211 -0.78575811 -0.608296    0.        ]
 [ 0.3907936  -0.59965591 -0.6983503   0.        ]
 [-0.00986732 -0.84406471 -0.53615054  0.        ]
 [ 0.39545306  0.66495687 -0.63360022  0.        ]
 [-0.00669149  0.4254636  -0.9049508   0.        ]
 [-0.16346177 -0.58432131 -0.79488921  0.        ]
 [ 0.23777039 -0.39254581 -0.88846668  0.        ]
 [ 0.61105143  0.75788678 -0.22852523  0.        ]
 [-0.28166398  0.66730551 -0.68946991  0.        ]
 [-0.21185811 -0.58018078 -0.78645178  0.        ]]


In [63]:
N = int(1e6)
velosity_matrix = np.zeros((N, 3))

typeID = np.zeros(N)
# FO_ratio = int(N/4)
# typeID[-FO_ratio:] = 1

ion_ration = int(N)
typeID[-ion_ration:] = 2
velosity_matrix[-ion_ration:, 0] = np.random.rand(ion_ration)*0.01 - 0.005
velosity_matrix[-ion_ration:, 1] = np.random.rand(ion_ration)*0.01 - 0.005
velosity_matrix[-ion_ration:, 2] = -1 

vel_type_shuffle = np.zeros((N, 4))
vel_type_shuffle[:, :3] = velosity_matrix
vel_type_shuffle[:, -1] = typeID

np.random.shuffle(vel_type_shuffle)
# print(typeID[:10])
# print(velosity_matrix[0])
print(vel_type_shuffle[:10])

[[ 5.40091043e-04  3.14177961e-03 -1.00000000e+00  2.00000000e+00]
 [-4.54078428e-03  2.57382055e-03 -1.00000000e+00  2.00000000e+00]
 [ 3.77557195e-03  3.94541595e-03 -1.00000000e+00  2.00000000e+00]
 [ 4.05191839e-03  1.63245413e-03 -1.00000000e+00  2.00000000e+00]
 [-2.26384230e-03 -3.41551823e-03 -1.00000000e+00  2.00000000e+00]
 [-2.50544706e-04  4.96622775e-04 -1.00000000e+00  2.00000000e+00]
 [ 2.16436864e-03  3.45842638e-03 -1.00000000e+00  2.00000000e+00]
 [ 1.74127788e-03  2.16614901e-03 -1.00000000e+00  2.00000000e+00]
 [ 2.57891242e-03 -2.19787644e-03 -1.00000000e+00  2.00000000e+00]
 [-3.60594991e-03 -2.53308143e-03 -1.00000000e+00  2.00000000e+00]]


In [14]:
print(vel_type_shuffle[10:20])

[[-3.68380774e-01  5.11607868e-01 -7.76242872e-01  0.00000000e+00]
 [-2.62083917e-01 -8.45568133e-01 -4.65109183e-01  0.00000000e+00]
 [-5.65620406e-01  3.02950370e-01 -7.67003670e-01  0.00000000e+00]
 [-6.15339422e-02 -4.23597030e-01 -9.03758336e-01  0.00000000e+00]
 [-4.84750871e-01  1.73537876e-01 -8.57263786e-01  0.00000000e+00]
 [-2.14957732e-01  5.32181143e-01 -8.18887297e-01  0.00000000e+00]
 [-3.23637043e-01 -7.59884259e-01 -5.63768550e-01  0.00000000e+00]
 [-4.28698360e-01 -1.71049598e-01 -8.87107520e-01  0.00000000e+00]
 [ 8.46669637e-04  4.75093596e-04 -1.00000000e+00  2.00000000e+00]
 [-5.71897263e-01  4.47035248e-01 -6.87817568e-01  0.00000000e+00]]


In [49]:
# c4f8

N = int(1e7)
velosity_matrix = np.zeros((N, 3))
# tstep=1e-5
# celllength=1e-5
# velosity_matrix[:, 0] = -1 * celllength /tstep
# velosity_matrix[:, 1] = -1 * celllength /tstep
Random1 = np.random.rand(N)
Random2 = np.random.rand(N)
Random3 = np.random.rand(N)
velosity_matrix = np.array([max_velocity_u(Random1, Random2), \
                            max_velocity_w(Random1, Random2), \
                                max_velocity_v(Random3)]).T

energy = np.linalg.norm(velosity_matrix, axis=1)
velosity_matrix[:,0] = np.divide(velosity_matrix[:,0], energy)
velosity_matrix[:,1] = np.divide(velosity_matrix[:,1], energy)
velosity_matrix[:,2] = np.divide(velosity_matrix[:,2], energy)

# velosity_matrix[:, 2] = -1 * celllength /tstep

typeID = np.ones(N)
# FO_ratio = int(N/4)
# typeID[-FO_ratio:] = 1

# ion_ration = int(N/8)
# typeID[-ion_ration:] = 2
# velosity_matrix[-ion_ration:, 0] = np.random.rand(ion_ration)*0.001
# velosity_matrix[-ion_ration:, 1] = np.random.rand(ion_ration)*0.001
# velosity_matrix[-ion_ration:, 2] = -1 

vel_type_shuffle = np.zeros((N, 4))
vel_type_shuffle[:, :3] = velosity_matrix
vel_type_shuffle[:, -1] = typeID

np.random.shuffle(vel_type_shuffle)
# print(typeID[:10])
# print(velosity_matrix[0])
print(vel_type_shuffle[:10])

[[-0.17839113 -0.86820551 -0.46302894  1.        ]
 [ 0.41936594  0.45306316 -0.78668036  1.        ]
 [ 0.68916596 -0.47394099 -0.54811516  1.        ]
 [-0.64269414 -0.07646823 -0.76229709  1.        ]
 [-0.76288849 -0.25496899 -0.59413127  1.        ]
 [-0.69749417  0.50898933 -0.50441228  1.        ]
 [ 0.17935686  0.28565881 -0.94139798  1.        ]
 [ 0.15481802  0.45571212 -0.87656023  1.        ]
 [ 0.5249166  -0.56691386 -0.63487891  1.        ]
 [ 0.28321522  0.69085135 -0.66521692  1.        ]]


In [10]:
#etching
logname = 'Multi_species_bosch_911'
testEtch = Multi_Species_bosch_etching.etching(mirror=True,inputMethod='bunch', depo_or_etching='etching', 
                            etchingPoint = np.array([center, center, 37]),depoPoint = np.array([center, center, 37]),
                            density=density, center_with_direction=np.array([[35,100,75]]), 
                            range3D=np.array([[0, 70, 0, 100, 0, 150]]), InOrOut=[1], yield_hist=np.array([None]),
                            reaction_type=False, param = [1.6, -0.7], N = N, 
                            sub_xy=[0,0], film=etchfilm, n=1, cellSize=etchfilm.shape, 
                            celllength=1e-5, kdtreeN=5, tstep=1e-5,
                            substrateTop=80,posGeneratorType='top', logname=logname)


# etching = testEtch.depo_position_increase_cosVel_normal(125, int(7e6), 2e-3, 5)


In [52]:
#depo
logname = 'Multi_species_bosch_911'
testEtch = Multi_Species_bosch_etching.etching(mirror=True,inputMethod='bunch', depo_or_etching='depo', 
                            etchingPoint = np.array([center, center, 45]),depoPoint = np.array([center, center, 45]),
                            density=density, center_with_direction=np.array([[35,100,75]]), 
                            range3D=np.array([[0, 70, 0, 100, 0, 150]]), InOrOut=[1], yield_hist=np.array([None]),
                            reaction_type=False, param = [1.6, -0.7], N = N, 
                            sub_xy=[0,0], film=etchfilm, n=1, cellSize=etchfilm.shape, 
                            celllength=1e-5, kdtreeN=5, tstep=1e-5,
                            substrateTop=80,posGeneratorType='top', logname=logname)

In [11]:
%lprun -f testEtch.inputParticle etching = testEtch.inputParticle(125,vel_type_shuffle[:,:3], vel_type_shuffle[:,-1], 2e-2, 5)

running:  42%|██████████████████████▎                              | 42.0/100 [26:53<37:08, 38.4s/B]

etch finish


Timer unit: 1e-07 s

Total time: 1614.26 s
File: d:\ysy\etching\Multi_Species_bosch_etching.py
Function: inputParticle at line 541

Line #      Hits         Time  Per Hit   % Time  Line Contents
   541                                               def inputParticle(self, randomSeed, velosity_matrix, typeID, tmax, Zgap):
   542         1        636.0    636.0      0.0          np.random.seed(randomSeed)
   543         1        2e+10    2e+10    100.0          result =  self.runEtch(velosity_matrix, typeID, tmax, emptyZ=Zgap)
   544                                                   # if np.any(result[0][:, :, self.depoThick]) != 0:
   545                                                   #     break             
   546         1         61.0     61.0      0.0          del self.log, self.fh 
   547         1          9.0      9.0      0.0          return result

In [70]:
etching = testEtch.inputParticle(125,vel_type_shuffle[:,:3], vel_type_shuffle[:,-1], 2e-2, 5)

running:  67%|███████████████████████████████████▌                 | 67.0/100 [43:54<21:37, 39.3s/B]


KeyboardInterrupt: 

In [13]:
np.save('bosch_sf_step11', etchfilm)

In [47]:
etchfilm[50, 50, 60:]

array([[10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [ 0.,  6.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,

In [13]:
etchfilm[50, 50, 60:]

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.

In [43]:
etching[0][50, 50, 60:]

array([[10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [10.,  0.],
       [ 0., 10.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,

In [12]:
depo1 = torch.Tensor(etching[0][:, 50:, :, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[0][:, 50:, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:62984/index.html?ui=P_0x2558df690a0_2&reconnect=auto" class="pyvis…

In [66]:
depo1 = torch.Tensor(etchfilm[:50, :, :, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etchfilm[:50, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:63391/index.html?ui=P_0x15f92cd3020_8&reconnect=auto" class="pyvis…